In [1]:
def getRanges(rangesString):
    ranges = rangesString.split(' or ')
    return [[int(rg.split('-')[0]), int(rg.split('-')[1])] for rg in ranges]
def getTicket(ticket):
    return [int(n) for n in ticket.split(',')]

In [2]:
fileInput = open('data.txt', mode='r')
dataInput = [group.split('\n') for group in fileInput.read().split('\n\n')]
fileInput.close()

filedsRulesInput = { rule.split(':')[0]: getRanges(rule.split(':')[1]) for rule in dataInput[0]}
myTicketInput = getTicket(dataInput[1][1])
nearbyTicketsInput = [ getTicket(t) for t in dataInput[2][1::]]

In [3]:
def isValidNumberOneRule(nb, rule):
    return (rule[0][0] <= nb <= rule[0][1]) or (rule[1][0] <= nb <= rule[1][1])

def isValidNumber(nb, rules):
    for rule in rules:
        if(isValidNumberOneRule(nb,rule)): return True
    return False

def findInvalidNumbersInTicket(ticket, rules):
    return [n for n in ticket if(not(isValidNumber(n,rules.values())))]

In [4]:
def deleteInvalidTicket(nearbyTickets, rules):
    return [ ticket for ticket in nearbyTickets if len(findInvalidNumbersInTicket(ticket,rules))== 0 ]

In [5]:
def findPositionsOfOneRuleInTicket(tickets, rule):
    ticketLength = len(tickets[0])
    possiblePositions = []
    for position in range(ticketLength):
        numbersToTest = [ticket[position] for ticket in tickets]
        boolRes = True
        for nb in numbersToTest:
            if(not(isValidNumberOneRule(nb,rule))):
                boolRes = False; break
        if(boolRes): possiblePositions.append(position)
    return possiblePositions

In [6]:
def getFoundRule(rulesPosition):
    for name,positions in rulesPosition.items():
        if len(positions) == 1: return (name,positions[0])

def removePossiblePosition(rulesPosition, position):
    newRules = {}
    for name,positions in rulesPosition.items():
        positions.remove(position)
        if(len(positions)>0): newRules[name] = positions
    return newRules

def findPossiblesPositionOfRules(tickets, rules):
    return { name: findPositionsOfOneRuleInTicket(tickets,rule) for name,rule in rules.items() }

def findPositionForRules(tickets, rules):
    res, rulesPos = {}, { name: findPositionsOfOneRuleInTicket(tickets,rule) for name,rule in rules.items() }
    while(len(rulesPos)>0):
        ruleFound = getFoundRule(rulesPos)
        res[ruleFound[0]] = ruleFound[1]
        rulesPos = removePossiblePosition(rulesPos,ruleFound[1])
    return res

In [7]:
from functools import reduce

def calculForPart2(nearbyTickets,fieldsRules,myTicket):
    validTickets = deleteInvalidTicket(nearbyTickets, fieldsRules)
    return [myTicket[position] for name,position in findPositionForRules(validTickets,fieldsRules).items() if 'departure' in name]

In [8]:
print('Part 1 :', sum([ sum(findInvalidNumbersInTicket(ticket, filedsRulesInput)) for ticket in nearbyTicketsInput]))
print('Part 2 :', reduce((lambda x,y : x*y),calculForPart2(nearbyTicketsInput,filedsRulesInput, myTicketInput)))

Part 1 : 20060
Part 2 : 2843534243843
